In [79]:
import pandas as pd
import numpy as np
from itertools import chain
from collections import Counter
import pandas as pd
import numpy as np
from itertools import chain
from collections import Counter
import textstat
import seaborn as sns
import matplotlib.pyplot as plt
# Set the style of the plot
sns.set_style('whitegrid')
plt.rcParams['font.family'] = ['SimHei']

import math
import re

from tqdm import tqdm
from sklearn.cluster import KMeans
import geopy

import warnings
warnings.filterwarnings('ignore')

from matplotlib import style
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', None)
%matplotlib inline
style.use("fivethirtyeight")

In [2]:
v202303_vote_within7days=pd.read_csv('data/v202403_vote_within7days.csv')
v202203_vote_within7days=pd.read_csv('data/v202303_vote_within7days.csv')

In [77]:
v202203_vote_within7days.columns

Index(['Id', 'VoteType1Count', 'VoteType2Count', 'VoteType3Count'], dtype='object')

In [78]:
v2023_posts_df=pd.read_pickle('data/240412-v2023_posts_df.pickle')
v2023_posts_df=v2023_posts_df.merge(v202303_vote_within7days,how='left',left_on='Id',right_on='Id')
v2023_posts_df[['VoteType1Count', 'VoteType2Count', 'VoteType3Count']]=v2023_posts_df[['VoteType1Count', 'VoteType2Count', 'VoteType3Count']].fillna(0)
v2023_posts_df


,Id,PostTypeId,AcceptedAnswerId,...,VoteType1Count,VoteType2Count,VoteType3Count
0,70177579,2,NaN,...,0.0,0.0,0.0
1,70177580,2,NaN,...,0.0,0.0,0.0
2,70177581,2,NaN,...,1.0,0.0,0.0
3,70177583,2,NaN,...,0.0,1.0,0.0
4,70177584,2,NaN,...,0.0,0.0,0.0
...,...,...,...,...,...,...,...
5645469,78091308,1,NaN,...,0.0,0.0,0.0
5645470,78091309,1,NaN,...,0.0,0.0,0.0
5645471,78091310,2,NaN,...,0.0,0.0,0.0
5645472,78091311,1,NaN,...,0.0,0.0,0.0


In [5]:
v2022_posts_df=pd.read_pickle('data/240412-v2022_posts_df.pickle')
v2022_posts_df=v2022_posts_df.merge(v202203_vote_within7days,how='left',left_on='Id',right_on='Id')
v2022_posts_df[['VoteType1Count', 'VoteType2Count', 'VoteType3Count']]=v2022_posts_df[['VoteType1Count', 'VoteType2Count', 'VoteType3Count']].fillna(0)
v2022_posts_df


,Id,PostTypeId,AcceptedAnswerId,...,VoteType1Count,VoteType2Count,VoteType3Count
0,65082871,2,NaN,...,1.0,3.0,0.0
1,65082872,2,NaN,...,0.0,0.0,0.0
2,65082874,1,65082893.0,...,0.0,1.0,0.0
3,65082876,2,NaN,...,0.0,0.0,0.0
4,65082878,1,NaN,...,0.0,1.0,0.0
...,...,...,...,...,...,...,...
7750266,75640478,1,NaN,...,0.0,0.0,0.0
7750267,75640479,2,NaN,...,0.0,1.0,0.0
7750268,75640480,1,NaN,...,0.0,0.0,0.0
7750269,75640481,1,NaN,...,0.0,0.0,0.0


In [6]:
v2023_posts_q_to_a_df=pd.read_pickle('data/240412-v2023_posts_q_to_a_df.pickle').drop_duplicates()
v2023_posts_q_to_a_df.columns=['Q_'+i for i in v2023_posts_q_to_a_df.columns]
v2023_posts_df=v2023_posts_df.merge(v2023_posts_q_to_a_df,how='left',left_on='ParentId',right_on='Q_Id')
del v2023_posts_q_to_a_df
v2023_posts_df


,Id,PostTypeId,AcceptedAnswerId,...,Q_pure_body_flesch_kincaid_grade,Q_pure_body_flesch_reading_ease,Q_pure_body_smog_index
0,70177579,2,NaN,...,2.8,94.35,8.1
1,70177580,2,NaN,...,7.1,67.04,10.0
2,70177581,2,NaN,...,9.1,61.87,11.2
3,70177583,2,NaN,...,5.9,70.09,9.4
4,70177584,2,NaN,...,4.4,84.78,8.8
...,...,...,...,...,...,...,...
5645469,78091308,1,NaN,...,NaN,NaN,NaN
5645470,78091309,1,NaN,...,NaN,NaN,NaN
5645471,78091310,2,NaN,...,2.8,88.84,6.8
5645472,78091311,1,NaN,...,NaN,NaN,NaN


In [7]:
v2023_posts_df.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
       'DeletionDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId',
       'OwnerDisplayName', 'LastEditorUserId', 'LastEditorDisplayName',
       'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount',
       'CommentCount', 'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate',
       'body_string_len', 'body_lexicon_count', 'body_flesch_kincaid_grade',
       'body_flesch_reading_ease', 'body_smog_index', 'code_string_len',
       'code_lexicon_count', 'samp_string_len', 'samp_lexicon_count',
       'kbd_lexicon_count', 'kbd_string_len', 'var_string_len',
       'var_lexicon_count', 'full_code_string_len', 'full_code_lexicon_count',
       'pure_body_string_len', 'pure_body_lexicon_count',
       'pure_body_flesch_kincaid_grade', 'pure_body_flesch_reading_ease',
       'pure_body_smog_index', 'VoteType1Count', 'VoteType2Count',
       'VoteType3Count', 'Q_Id', 'Q_PostTypeId', 'Q_AcceptedAnswerId',
 

In [8]:
v2022_posts_q_to_a_df=pd.read_pickle('data/240412-v2022_posts_q_to_a_df.pickle').drop_duplicates()
v2022_posts_q_to_a_df.columns=['Q_'+i for i in v2022_posts_q_to_a_df.columns]
v2022_posts_df=v2022_posts_df.merge(v2022_posts_q_to_a_df,how='left',left_on='ParentId',right_on='Q_Id')
del v2022_posts_q_to_a_df
v2022_posts_df

,Id,PostTypeId,AcceptedAnswerId,...,Q_pure_body_flesch_kincaid_grade,Q_pure_body_flesch_reading_ease,Q_pure_body_smog_index
0,65082871,2,NaN,...,12.6,63.36,0.0
1,65082872,2,NaN,...,4.3,79.56,8.3
2,65082874,1,65082893.0,...,NaN,NaN,NaN
3,65082876,2,NaN,...,7.9,64.91,10.7
4,65082878,1,NaN,...,NaN,NaN,NaN
...,...,...,...,...,...,...,...
7750266,75640478,1,NaN,...,NaN,NaN,NaN
7750267,75640479,2,NaN,...,3.5,92.53,8.3
7750268,75640480,1,NaN,...,NaN,NaN,NaN
7750269,75640481,1,NaN,...,NaN,NaN,NaN


In [9]:
v2023_posts_df.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
       'DeletionDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId',
       'OwnerDisplayName', 'LastEditorUserId', 'LastEditorDisplayName',
       'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount',
       'CommentCount', 'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate',
       'body_string_len', 'body_lexicon_count', 'body_flesch_kincaid_grade',
       'body_flesch_reading_ease', 'body_smog_index', 'code_string_len',
       'code_lexicon_count', 'samp_string_len', 'samp_lexicon_count',
       'kbd_lexicon_count', 'kbd_string_len', 'var_string_len',
       'var_lexicon_count', 'full_code_string_len', 'full_code_lexicon_count',
       'pure_body_string_len', 'pure_body_lexicon_count',
       'pure_body_flesch_kincaid_grade', 'pure_body_flesch_reading_ease',
       'pure_body_smog_index', 'VoteType1Count', 'VoteType2Count',
       'VoteType3Count', 'Q_Id', 'Q_PostTypeId', 'Q_AcceptedAnswerId',
 

In [10]:
v2023_posts_base_df=v2023_posts_df[(v2023_posts_df['PostTypeId']==2)
                            &(v2023_posts_df['OwnerUserId'].notnull())
                              &(v2023_posts_df['CreationDate']>='2021-12-01')
                              &(v2023_posts_df['CreationDate']<='2023-12-01')]

In [11]:
v2022_posts_base_df=v2022_posts_df[(v2022_posts_df['PostTypeId']==2)
                                  &(v2022_posts_df['OwnerUserId'].notnull())
                                  &(v2022_posts_df['CreationDate']>='2020-12-01')
                                  &(v2022_posts_df['CreationDate']<='2022-12-01')]

In [12]:
v2022_posts_base_df['ans_ques_time(h)']=(v2022_posts_base_df['CreationDate']-v2022_posts_base_df['Q_CreationDate']).apply(lambda x:x.seconds/3600)
v2023_posts_base_df['ans_ques_time(h)']=(v2023_posts_base_df['CreationDate']-v2023_posts_base_df['Q_CreationDate']).apply(lambda x:x.seconds/3600)


In [13]:
import pickle
with open("location_nation.pkl",'rb') as f:      # 读文件
    location_nation = pickle.load(f) 
with open("dict_Europen_nations.pkl",'rb') as f:      # 读文件
    dict_Europen_nations = pickle.load(f) 
dict_Europen_nations

{'Russia': 'Россия',
 'Ukraine': 'Україна',
 'France': 'France',
 'Spain': 'España',
 'Sweden': 'Sverige',
 'Germany': 'Deutschland',
 'Finland': 'Suomi / Finland',
 'Norway': 'Norge',
 'Poland': 'Polska',
 'Italy': 'Italia',
 'United Kingdom': 'United Kingdom',
 'Romania': 'România',
 'Belarus': 'Беларусь',
 'Greece': 'Ελλάς',
 'Bulgaria': 'България',
 'Iceland': 'Ísland',
 'Portugal': 'Portugal',
 'Czech Republic': 'Česko',
 'Denmark': 'Danmark',
 'Hungary': 'Magyarország',
 'Serbia': 'Србија',
 'Austria': 'Österreich',
 'Ireland': 'Éire / Ireland',
 'Lithuania': 'Lietuva',
 'Latvia': 'Latvija',
 'Croatia': 'Hrvatska',
 'Bosnia and Herzegovina': 'Bosna i Hercegovina / Босна и Херцеговина',
 'Slovakia': 'Slovensko',
 'Estonia': 'Eesti',
 'Netherlands': 'Nederland',
 'Switzerland': 'Schweiz/Suisse/Svizzera/Svizra',
 'Moldova': 'Moldova',
 'Belgium': 'België / Belgique / Belgien',
 'Albania': 'Shqipëria',
 'Macedonia': 'Северна Македонија',
 'Slovenia': 'Slovenija',
 'Montenegro': 'Crna

In [14]:
dict_nations_continent={k:'Europen' for k in dict_Europen_nations.keys()}
dict_nations_continent['United States']='United States'

In [15]:
dict_nations={v:k for k,v in dict_Europen_nations.items()}
dict_nations['United States']='United States'

In [16]:
df_address=pd.DataFrame(location_nation,index=['address','point']).T
df_address

,address,point
on the server farm,None,None
"El Cerrito, CA","El Cerrito, Contra Costa County, CAL Fire Nort...","(37.9154056, -122.301411)"
"Corvallis, OR","Corvallis, Benton County, Oregon, United States","(44.5645659, -123.2620435)"
"Johnson City, TN, USA","Johnson City, Washington County, East Tennesse...","(36.3134398, -82.3534728)"
"New York, NY","City of New York, New York, United States","(40.7127281, -74.0060152)"
...,...,...
"Lochau, Vorarlberg, Austria","Gemeinde Lochau, Bezirk Bregenz, Vorarlberg, 6...","(47.5294182, 9.7533944)"
"Miramar, FL","Miramar, Broward County, Florida, United States","(25.9873137, -80.2322706)"
"Hefei, Anhui, China (中国 安徽省 合肥市)","市三院, 望江东路, 东华科技股份有限公司, 芜湖路街道, 淝河镇, 包河区, 安徽省, 2...","(31.8307479, 117.26474205000002)"
"Israel, Tel-Aviv","תל אביב-יפו, נפת תל אביב, מחוז תל אביב, ישראל","(32.0852997, 34.7818064)"


In [17]:
v2023_users=pd.read_pickle('data/users_in_v202403.pickle')
v2023_users.columns=['User_'+i for i in v2023_users.columns]

v2022_users=pd.read_pickle('data/users_in_v202303.pickle')
v2022_users.columns=['User_'+i for i in v2022_users.columns]

In [18]:
len(v2023_users['User_Id'].unique())

1759125

In [19]:
df_states=pd.read_csv('data/states.csv')
def get_nation(x):
    if type(x) is str:
        return x.split(', ')[-1]
def get_state(x):
    if type(x) is str and 'United States' in x:
        for state in df_states.State:
            if state in x:
                return state

In [20]:
v2023_users=v2023_users[v2023_users['User_Id'].isin(v2023_posts_base_df['OwnerUserId'])]
v2023_users=v2023_users[v2023_users['User_Location'].notnull()]
v2023_users=v2023_users.merge(df_address,how='left',left_on='User_Location',right_index=True, validate = 'm:1')

v2022_users=v2022_users[v2022_users['User_Id'].isin(v2022_posts_base_df['OwnerUserId'])]
v2022_users=v2022_users[v2022_users['User_Location'].notnull()]
v2022_users=v2022_users.merge(df_address,how='left',left_on='User_Location',right_index=True, validate = 'm:1')


In [21]:
v2022_users.columns

Index(['User_Id', 'User_Reputation', 'User_CreationDate', 'User_DisplayName',
       'User_LastAccessDate', 'User_WebsiteUrl', 'User_Location',
       'User_AboutMe', 'User_Views', 'User_UpVotes', 'User_DownVotes',
       'User_ProfileImageUrl', 'User_EmailHash', 'User_AccountId', 'address',
       'point'],
      dtype='object')

In [22]:
v2022_users['User_Age']=(pd.to_datetime('2020-12-01')-v2022_users['User_CreationDate']).apply(lambda x:(x.days)/365)
v2023_users['User_Age']=(pd.to_datetime('2021-12-01')-v2023_users['User_CreationDate']).apply(lambda x:(x.days)/365)

v2022_users['User_Duration']=(v2022_users['User_LastAccessDate']-v2022_users['User_CreationDate']).apply(lambda x:(x.days)/365)
v2023_users['User_Duration']=(v2023_users['User_LastAccessDate']-v2023_users['User_CreationDate']).apply(lambda x:(x.days)/365)


In [23]:
v2023_users['nation']=v2023_users['address'].apply(lambda x:get_nation(x))
v2023_users['state']=v2023_users['address'].apply(lambda x:get_state(x))

v2022_users['nation']=v2022_users['address'].apply(lambda x:get_nation(x))
v2022_users['state']=v2022_users['address'].apply(lambda x:get_state(x))

In [24]:
v2023_posts_base_df=v2023_posts_base_df.merge(v2023_users[['User_Id','User_Reputation', 'User_LastAccessDate', 'User_CreationDate', 
    'User_Views', 'User_UpVotes', 'User_DownVotes','User_Age','User_Duration', 'address','point','nation','state',]],how='left',left_on='OwnerUserId',
                                              right_on='User_Id', validate = 'm:1')
v2022_posts_base_df=v2022_posts_base_df.merge(v2022_users[['User_Id','User_Reputation', 'User_LastAccessDate', 'User_CreationDate', 
    'User_Views', 'User_UpVotes', 'User_DownVotes', 'User_Age','User_Duration','address','point','nation','state']],how='left',left_on='OwnerUserId',
                                              right_on='User_Id', validate = 'm:1')

In [25]:
v2023_posts_base_df['nation']=v2023_posts_base_df['address'].apply(lambda x:get_nation(x))
v2023_posts_base_df['state']=v2023_posts_base_df['address'].apply(lambda x:get_state(x))

v2022_posts_base_df['nation']=v2022_posts_base_df['address'].apply(lambda x:get_nation(x))
v2022_posts_base_df['state']=v2022_posts_base_df['address'].apply(lambda x:get_state(x))

In [26]:
v2023_posts_base_df['full_code_string_ratio']=v2023_posts_base_df['full_code_string_len']/v2023_posts_base_df['body_string_len']

v2022_posts_base_df['full_code_string_ratio']=v2022_posts_base_df['full_code_string_len']/v2022_posts_base_df['body_string_len']

In [27]:
df_language=pd.read_table('main programming language.txt',header=None)
df_language=df_language.rename({0:'Language'},axis=1)
df_language['Language']=df_language['Language'].str.lower()
list_language=df_language['Language'].tolist()
list_language[0:5]

['python', 'java', 'c++', 'c', 'javascript']

In [28]:
v2022_posts_base_df['Q_Tags'] = v2022_posts_base_df['Q_Tags'].fillna('')
v2023_posts_base_df['Q_Tags'] = v2023_posts_base_df['Q_Tags'].fillna('')

v2022_posts_base_df['Q_Tags_list']=v2022_posts_base_df['Q_Tags'].apply(lambda x:re.findall(r"<(.*?)>", x))
v2023_posts_base_df['Q_Tags_list']=v2023_posts_base_df['Q_Tags'].apply(lambda x:re.findall(r"<(.*?)>", x))

v2022_posts_base_df['Q_Tags_list_filter']=v2022_posts_base_df['Q_Tags_list'].apply(lambda x:[i for i in x if i in list_language])
v2023_posts_base_df['Q_Tags_list_filter']=v2023_posts_base_df['Q_Tags_list'].apply(lambda x:[i for i in x if i in list_language])

v2022_posts_base_df['Q_Tags_len']=v2022_posts_base_df['Q_Tags_list'].apply(lambda x:len(x))
v2023_posts_base_df['Q_Tags_len']=v2023_posts_base_df['Q_Tags_list'].apply(lambda x:len(x))

v2022_posts_base_df['Q_Tags_list_filter_len']=v2022_posts_base_df['Q_Tags_list_filter'].apply(lambda x:len(x))
v2023_posts_base_df['Q_Tags_list_filter_len']=v2023_posts_base_df['Q_Tags_list_filter'].apply(lambda x:len(x))

In [99]:
print('Nation Number 2023--',len(Counter(v2023_posts_base_df['nation'])))
print('Users Number 2023--',len(v2023_posts_base_df['OwnerUserId'].unique()))

Counter(v2023_posts_base_df['nation']).most_common(n=20)

Nation Number 2023-- 225
Users Number 2023-- 686680


[(None, 1275144),
 ('United States', 323303),
 ('India', 161115),
 ('Deutschland', 109754),
 ('United Kingdom', 102426),
 ('France', 61998),
 ('Canada', 51723),
 ('Australia', 43234),
 ('Nederland', 43029),
 ('Polska', 27443),
 ('Italia', 24175),
 ('پاکستان', 23016),
 ('ایران', 22643),
 ('Россия', 21774),
 ('Schweiz/Suisse/Svizzera/Svizra', 21468),
 ('España', 21229),
 ('ישראל', 20983),
 ('Türkiye', 20634),
 ('Österreich', 20616),
 ('中国', 20537)]

In [98]:
print('Nation Number 2022--',len(Counter(v2022_posts_base_df['nation'])))
print('Users Number 2022--',len(v2022_posts_base_df['OwnerUserId'].unique()))

Counter(v2022_posts_base_df['nation']).most_common(n=20)

Nation Number 2022-- 234
Nation Number 2022-- 788241


[(None, 1512183),
 ('United States', 455775),
 ('India', 263028),
 ('Deutschland', 150243),
 ('United Kingdom', 141681),
 ('France', 84114),
 ('Canada', 71971),
 ('Nederland', 61571),
 ('Australia', 55335),
 ('Polska', 41689),
 ('Россия', 36051),
 ('Italia', 35026),
 ('ایران', 33501),
 ('پاکستان', 32380),
 ('España', 31897),
 ('Österreich', 30272),
 ('বাংলাদেশ', 30224),
 ('ישראל', 30133),
 ('中国', 29927),
 ('Brasil', 28950)]

In [31]:
list_meaning_nation_2023=[k for k,v in Counter(v2023_posts_base_df['nation']).items() if v>20]
list_meaning_nation_2022=[k for k,v in Counter(v2022_posts_base_df['nation']).items() if v>20]
list_meaning_nation=[i for i in list_meaning_nation_2022 if i in list_meaning_nation_2023]
mean_list=['Africa','Ōu',None,'Soul Buoy','中国','Россия','Україна','Italia','조선민주주의인민공화국',
           'ایران','سورية, سوريا','香港','Hong Kong','Cuba']
list_meaning_nation=[i for i in list_meaning_nation_2022 if i not in mean_list]
len(list_meaning_nation)
# Africa

170

In [32]:
# list_meaning_nation_2023=[v for k,v in Counter(v2023_posts_base_df['nation']).items() if v>20]
# list_meaning_nation_2022=[v for k,v in Counter(v2022_posts_base_df['nation']).items() if v>20]

# User-Nation

In [33]:
df_v2023_comment=pd.read_pickle('data/df_v2023_comment.pickle')
df_v2022_comment=pd.read_pickle('data/df_v2022_comment.pickle')


In [34]:
df_v2022_comment

,PostId,CommentScore,TotalComment,AnswerComment,QuestionerComment,ViewerComment
0,65082871,0.5,2,1.0,1.0,0.0
1,65082890,0.0,1,0.0,1.0,0.0
2,65082895,0.0,1,0.0,1.0,0.0
3,65082899,0.0,1,0.0,1.0,0.0
4,65082903,0.0,2,1.0,1.0,0.0
...,...,...,...,...,...,...
743042,74635428,0.0,2,1.0,1.0,0.0
743043,74635444,0.0,1,0.0,1.0,0.0
743044,74635446,0.0,1,0.0,1.0,0.0
743045,74635463,0.0,2,0.0,1.0,1.0


In [35]:
v2022_nation_post=v2022_posts_base_df[(v2022_posts_base_df['nation'].isin(list_meaning_nation))]
v2022_nation_post=v2022_nation_post.merge(df_v2022_comment,how='left',left_on='Id',right_on='PostId')
v2022_nation_post[[ 'CommentScore', 'TotalComment', 'AnswerComment', 'QuestionerComment', 'ViewerComment']]=v2022_nation_post[[ 'CommentScore', 'TotalComment', 'AnswerComment', 'QuestionerComment', 'ViewerComment']].fillna(0)
v2022_user_nation_post_group=v2022_nation_post.groupby(['User_Id',pd.Grouper(key='CreationDate', freq='D')]).agg(

     Up_vote = ('VoteType2Count','sum'),
     Down_vote = ('VoteType3Count','sum'),
    Score  =('Score', 'mean'),

    body_smog_index=('body_smog_index',  'mean'),
    pure_body_string_len =('pure_body_string_len', 'mean'),
    pure_body_smog_index =('pure_body_smog_index','mean'),

    Q_Score  =('Q_Score', 'mean'),
    Q_ViewCount =('Q_ViewCount', 'mean'),
    Q_AnswerCount  =('Q_AnswerCount', 'mean'),
    Q_CommentCount =('Q_CommentCount', 'mean'),
    Q_FavoriteCount =('Q_FavoriteCount', 'mean'),
    Q_body_string_len =('Q_body_string_len', 'mean'),

    ans_ques_time  =( 'ans_ques_time(h)', 'mean'),

    full_code_string_ratio =('full_code_string_ratio', 'mean'), 
     
    Q_Tags_set_filter =( 'Q_Tags_list_filter', lambda x:list(set(chain(*x)))),   

     TotalComment=('TotalComment',  'sum'),
     AnswerComment=('AnswerComment', 'sum'),
     QuestionerComment=('QuestionerComment', 'sum'),
     ViewerComment=('ViewerComment', 'sum'),
                                                            ).reset_index()

v2022_user_nation_post_group=v2022_user_nation_post_group.merge(v2022_users[['User_Id','nation']],how='left',left_on='User_Id',right_on='User_Id')
v2022_user_nation_post_group['version']='2022version'
v2022_user_nation_post_group['CreationDate_parallel']=v2022_user_nation_post_group['CreationDate']
v2022_user_nation_post_group['Delta_years']=(v2022_user_nation_post_group['CreationDate'].apply(lambda x:x.year) - 2020)
v2022_user_nation_post_group['year']=v2022_user_nation_post_group['CreationDate'].apply(lambda x:x.year)
v2022_user_nation_post_group['month']=v2022_user_nation_post_group['CreationDate'].apply(lambda x:x.month)
v2022_user_nation_post_group['day']=v2022_user_nation_post_group['CreationDate'].apply(lambda x:x.day)
v2022_user_nation_post_group['week_number']=v2022_user_nation_post_group['CreationDate'].apply(lambda x:x.isocalendar().week)
v2022_user_nation_post_group['week_day']=v2022_user_nation_post_group['CreationDate'].apply(lambda x:x.weekday())
v2022_user_nation_post_group['year_day']=v2022_user_nation_post_group['CreationDate'].apply(lambda x:x.timetuple().tm_yday)

v2022_user_nation_post_group['day_nb']=v2022_user_nation_post_group['year_day']+v2022_user_nation_post_group['Delta_years']*365
# v2022_user_nation_post_group['week_nb']=v2022_user_nation_post_group['year_day']+v2022_user_nation_post_group['Delta_years']*365
v2022_user_nation_post_group['month_nb']=v2022_user_nation_post_group['month']+v2022_user_nation_post_group['Delta_years']*12

v2022_user_nation_post_group.to_csv('pans/0606-v2022_user_nation_post_group.csv',encoding='utf-8-sig')
v2022_user_nation_post_group.to_pickle('pans/0606-v2022_user_nation_post_group.pickle')
# v2023_nation_post_group[v2023_nation_post_group['nation']=='대한민국'][['month_nb']]
v2022_user_nation_post_group

,User_Id,CreationDate,Up_vote,...,year_day,day_nb,month_nb
0,22.0,2021-10-21,0.0,...,294,659,22
1,22.0,2022-09-01,3.0,...,244,974,33
2,22.0,2022-09-02,2.0,...,245,975,33
3,33.0,2021-01-20,0.0,...,20,385,13
4,33.0,2021-02-18,0.0,...,49,414,14
...,...,...,...,...,...,...,...
1379936,21038281.0,2021-02-28,0.0,...,59,424,14
1379937,21038281.0,2021-06-22,0.0,...,173,538,18
1379938,21038281.0,2021-10-18,0.0,...,291,656,22
1379939,21038281.0,2021-12-13,0.0,...,347,712,24


In [36]:
v2023_nation_post=v2023_posts_base_df[(v2023_posts_base_df['nation'].isin(list_meaning_nation))]
v2023_nation_post=v2023_nation_post.merge(df_v2023_comment,how='left',left_on='Id',right_on='PostId')
v2023_nation_post[[ 'CommentScore', 'TotalComment', 'AnswerComment', 'QuestionerComment', 'ViewerComment']]=v2023_nation_post[[ 'CommentScore', 'TotalComment', 'AnswerComment', 'QuestionerComment', 'ViewerComment']].fillna(0)
v2023_user_nation_post_group=v2023_nation_post.groupby(['User_Id',pd.Grouper(key='CreationDate', freq='D')]).agg(
     Up_vote = ('VoteType2Count','sum'),
     Down_vote = ('VoteType3Count','sum'),
    Score  =('Score', 'mean'),

    body_smog_index=('body_smog_index',  'mean'),
    pure_body_string_len =('pure_body_string_len', 'mean'),
    pure_body_smog_index =('pure_body_smog_index','mean'),

    Q_Score  =('Q_Score', 'mean'),
    Q_ViewCount =('Q_ViewCount', 'mean'),
    Q_AnswerCount  =('Q_AnswerCount', 'mean'),
    Q_CommentCount =('Q_CommentCount', 'mean'),
    Q_FavoriteCount =('Q_FavoriteCount', 'mean'),
    Q_body_string_len =('Q_body_string_len', 'mean'),

    ans_ques_time  =( 'ans_ques_time(h)', 'mean'),

    full_code_string_ratio =('full_code_string_ratio', 'mean'), 
     
    Q_Tags_set_filter =( 'Q_Tags_list_filter', lambda x:list(set(chain(*x)))),   

     TotalComment=('TotalComment',  'sum'),
     AnswerComment=('AnswerComment', 'sum'),
     QuestionerComment=('QuestionerComment', 'sum'),
     ViewerComment=('ViewerComment', 'sum'),
                                                            ).reset_index()

v2023_user_nation_post_group=v2023_user_nation_post_group.merge(v2023_users[['User_Id','nation']],how='left',left_on='User_Id',right_on='User_Id')
v2023_user_nation_post_group['version']='2023version'
v2023_user_nation_post_group['CreationDate_parallel']=v2023_user_nation_post_group['CreationDate'] - pd.DateOffset(years=1)
v2023_user_nation_post_group['Delta_years']=(v2023_user_nation_post_group['CreationDate'].apply(lambda x:x.year) - 2021)
v2023_user_nation_post_group['year']=v2023_user_nation_post_group['CreationDate'].apply(lambda x:x.year)
v2023_user_nation_post_group['month']=v2023_user_nation_post_group['CreationDate'].apply(lambda x:x.month)
v2023_user_nation_post_group['day']=v2023_user_nation_post_group['CreationDate'].apply(lambda x:x.day)
v2023_user_nation_post_group['week_number']=v2023_user_nation_post_group['CreationDate'].apply(lambda x:x.isocalendar().week)
v2023_user_nation_post_group['week_day']=v2023_user_nation_post_group['CreationDate'].apply(lambda x:x.weekday())
v2023_user_nation_post_group['year_day']=v2023_user_nation_post_group['CreationDate'].apply(lambda x:x.timetuple().tm_yday)

v2023_user_nation_post_group['day_nb']=v2023_user_nation_post_group['year_day']+v2023_user_nation_post_group['Delta_years']*365
# v2023_nation_post_group['week_nb']=v2023_nation_post_group['year_day']+v2023_nation_post_group['Delta_years']*365
v2023_user_nation_post_group['month_nb']=v2023_user_nation_post_group['month']+v2023_user_nation_post_group['Delta_years']*12

# v2023_user_nation_post_group=v2023_user_nation_post_group.merge(v2023_posts_type1_day_df,how='left',left_on='CreationDate',right_on='CreationDate')
# v2023_user_nation_post_group=v2023_user_nation_post_group.merge(v2023_posts_type2_day_df,how='left',left_on='CreationDate',right_on='CreationDate')
v2023_user_nation_post_group.to_csv('pans/0606-v2023_user_nation_post_group.csv',encoding='utf-8-sig')
v2023_user_nation_post_group.to_pickle('pans/0606-v2023_user_nation_post_group.pickle')
# v2023_nation_post_group[v2023_nation_post_group['nation']=='대한민국'][['month_nb']]
v2023_user_nation_post_group

,User_Id,CreationDate,Up_vote,...,year_day,day_nb,month_nb
0,22.0,2022-09-01,4.0,...,244,609,21
1,22.0,2022-09-02,2.0,...,245,610,21
2,29.0,2023-03-03,0.0,...,62,792,27
3,33.0,2022-01-11,0.0,...,11,376,13
4,33.0,2022-04-04,0.0,...,94,459,16
...,...,...,...,...,...,...,...
1011358,23485659.0,2023-09-18,1.0,...,261,991,33
1011359,23485659.0,2023-09-24,0.0,...,267,997,33
1011360,23485659.0,2023-10-03,1.0,...,276,1006,34
1011361,23485659.0,2023-11-03,1.0,...,307,1037,35


In [37]:
df_user_nation_treat=pd.concat([v2022_user_nation_post_group,v2023_user_nation_post_group])
df_user_nation_treat['version_User_Id']=df_user_nation_treat['version']+'_'+df_user_nation_treat['User_Id'].astype(int).astype(str)

df_user_nation_treat['after']=np.where(df_user_nation_treat['CreationDate_parallel']>'2021-11-30',1,0)
df_user_nation_treat['treat']=np.where(df_user_nation_treat['version']=='2023version',1,0)
df_user_nation_treat=df_user_nation_treat[df_user_nation_treat['Score']>=0]
df_user_nation_treat=df_user_nation_treat[df_user_nation_treat['Q_Score']>=0]

df_user_nation_treat.to_csv('pans/0606-df_user_nation_treat.csv',encoding='utf-8-sig')
df_user_nation_treat.to_pickle('pans/0606-df_user_nation_treat.pickle')
df_user_nation_treat

,User_Id,CreationDate,Up_vote,...,version_User_Id,after,treat
0,22.0,2021-10-21,0.0,...,2022version_22,0,0
1,22.0,2022-09-01,3.0,...,2022version_22,1,0
2,22.0,2022-09-02,2.0,...,2022version_22,1,0
3,33.0,2021-01-20,0.0,...,2022version_33,0,0
4,33.0,2021-02-18,0.0,...,2022version_33,0,0
...,...,...,...,...,...,...,...
1011358,23485659.0,2023-09-18,1.0,...,2023version_23485659,1,1
1011359,23485659.0,2023-09-24,0.0,...,2023version_23485659,1,1
1011360,23485659.0,2023-10-03,1.0,...,2023version_23485659,1,1
1011361,23485659.0,2023-11-03,1.0,...,2023version_23485659,1,1


In [38]:
dict_month_nb_3m={12:1,13:1,14:1,15:2,16:2,17:2,18:3,19:3,20:3,21:4,22:4,23:4,
                 24:5,25:5,26:5,27:6,28:6,29:6,30:7,31:7,32:7,33:8,34:8,35:8}

In [70]:
df_user_nation_treat_m=df_user_nation_treat.groupby(['version_User_Id','month_nb','User_Id','nation']).agg(
    Up_vote = ('Up_vote', 'mean'),
    Down_vote = ('Down_vote', 'mean'),
    Score =('Score', 'mean'),
    body_smog_index= ('body_smog_index', 'mean'),
    pure_body_string_len =('pure_body_string_len', 'mean'),
    pure_body_smog_index =('pure_body_smog_index','mean'),
    Q_Score= ('Q_Score', 'mean'),
    Q_ViewCount= ('Q_ViewCount', 'mean'),
    Q_AnswerCount= ('Q_AnswerCount','mean'),
    Q_CommentCount= ('Q_CommentCount','mean'),
    Q_FavoriteCount= ('Q_FavoriteCount', 'mean'),
    Q_body_string_len= ('Q_body_string_len', 'mean'),
    ans_ques_time= ('ans_ques_time','mean'),
    full_code_string_ratio= ('full_code_string_ratio','mean'),
    Q_Tags_set_filter =( 'Q_Tags_set_filter', lambda x:list(set(chain(*x)))),      
     TotalComment=('TotalComment',  'sum'),
     AnswerComment=('AnswerComment', 'sum'),
     QuestionerComment=('QuestionerComment', 'sum'),
     ViewerComment=('ViewerComment', 'sum'),
            year=('year','max'), 
            month=('month','max'), 
            after=('after','max'), 
            treat=('treat','max')
                        ).reset_index()


df_user_nation_treat_m['version']=df_user_nation_treat_m['version_User_Id'].map(lambda x:x.split('_')[0])
df_user_nation_treat_m['month_nb_3m']=df_user_nation_treat_m['month_nb'].map(dict_month_nb_3m)

df_user_nation_treat_m['Q_Tags_len_filter']=df_user_nation_treat_m['Q_Tags_set_filter'].map(lambda x:len(x))


median_Q_Tags_len_filter = df_user_nation_treat_m.groupby('month_nb')['Q_Tags_len_filter'].transform('median')

df_user_nation_treat_m['median_Q_Tags_len_filter'] = (df_user_nation_treat_m['Q_Tags_len_filter'] > median_Q_Tags_len_filter).astype(int)
# df_user_nation_treat_m = df_user_nation_treat_m.groupby('version_User_Id').filter(lambda x: set(['2022version','2023version']).issubset(x['version']))

df_user_nation_treat_m.to_csv('pans/0606-df_user_nation_treat_m.csv',encoding='utf-8-sig')
df_user_nation_treat_m.to_pickle('pans/0606-df_user_nation_treat_m.pickle')
df_user_nation_treat_m

,version_User_Id,month_nb,User_Id,...,month_nb_3m,Q_Tags_len_filter,median_Q_Tags_len_filter
0,2022version_100,26,100.0,...,5,0,0
1,2022version_10000051,30,10000051.0,...,7,0,0
2,2022version_10000051,31,10000051.0,...,7,1,0
3,2022version_10000066,32,10000066.0,...,7,1,0
4,2022version_10000173,25,10000173.0,...,5,0,0
...,...,...,...,...,...,...,...
1142560,2023version_9999810,31,9999810.0,...,7,0,0
1142561,2023version_9999810,32,9999810.0,...,7,1,0
1142562,2023version_9999810,33,9999810.0,...,8,0,0
1142563,2023version_999986,15,999986.0,...,2,1,0


In [95]:
df_user_nation_treat_m_save=df_user_nation_treat_m[df_user_nation_treat_m['month_nb'].isin(range(12, 36))]

df_user_nation_treat_m_save=df_user_nation_treat_m_save.groupby('version_User_Id').filter(lambda x: set(range(12, 36)).issubset(x['month_nb']))
df_user_nation_treat_m_save=df_user_nation_treat_m_save.groupby('User_Id').filter(lambda x: set(['2022version','2023version']).issubset(x['version']))
df_user_nation_treat_m_save.to_csv('pans/0606-df_user_nation_treat_m_save.csv',encoding='utf-8-sig')

df_user_nation_treat_m_save

,version_User_Id,month_nb,User_Id,nation,Up_vote,Down_vote,Score,body_smog_index,pure_body_string_len,pure_body_smog_index,Q_Score,Q_ViewCount,Q_AnswerCount,Q_CommentCount,Q_FavoriteCount,Q_body_string_len,ans_ques_time,full_code_string_ratio,Q_Tags_set_filter,TotalComment,AnswerComment,QuestionerComment,ViewerComment,year,month,after,treat,version,month_nb_3m,Q_Tags_len_filter,median_Q_Tags_len_filter
7815,2022version_10195153,12,10195153.0,Australia,0.666667,0.0,0.416667,8.075000,311.000000,7.825000,0.500000,378.916667,1.500000,1.083333,0.0,1105.583333,2.857639,0.141767,"[python, ruby, go, kotlin]",8.0,2.0,5.0,1.0,2020,12,0,0,2022version,1,4,1
7816,2022version_10195153,13,10195153.0,Australia,0.250000,0.0,0.250000,5.375000,158.750000,2.987500,0.250000,374.125000,1.250000,1.250000,NaN,3005.125000,1.172257,0.359383,"[c, python, php, javascript]",5.0,1.0,4.0,0.0,2021,1,0,0,2022version,1,4,1
7817,2022version_10195153,14,10195153.0,Australia,1.625000,0.0,1.250000,7.897917,245.708333,7.466667,0.541667,1046.791667,2.166667,2.354167,0.0,935.312500,2.165260,0.353198,[php],14.0,5.0,9.0,0.0,2021,2,0,0,2022version,1,1,0
7818,2022version_10195153,15,10195153.0,Australia,0.666667,0.0,0.666667,10.600000,463.333333,9.500000,0.333333,291.666667,1.666667,1.000000,NaN,852.333333,1.480926,0.393182,[sql],12.0,4.0,6.0,2.0,2021,3,0,0,2022version,2,1,0
7819,2022version_10195153,16,10195153.0,Australia,1.000000,0.0,1.000000,6.440000,326.700000,5.580000,0.600000,516.900000,1.200000,1.000000,0.0,1178.700000,1.704333,0.212420,"[php, sql, c#]",9.0,2.0,6.0,1.0,2021,4,0,0,2022version,2,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141766,2023version_997358,31,997358.0,United States,0.625000,0.0,0.458333,7.820833,356.458333,6.425000,0.583333,101.541667,2.333333,1.916667,NaN,1315.416667,8.486250,0.389337,[],5.0,1.0,3.0,1.0,2023,7,1,1,2023version,7,0,0
1141767,2023version_997358,32,997358.0,United States,1.600000,0.0,1.600000,7.080000,290.266667,5.373333,0.733333,98.733333,2.866667,1.666667,NaN,1491.133333,5.452296,0.447730,[shell],4.0,0.0,1.0,3.0,2023,8,1,1,2023version,7,1,0
1141768,2023version_997358,33,997358.0,United States,1.000000,0.0,1.000000,12.950000,355.000000,7.050000,0.500000,128.000000,3.500000,0.000000,NaN,1553.000000,5.619444,0.475924,[],4.0,2.0,1.0,1.0,2023,9,1,1,2023version,8,0,0
1141769,2023version_997358,34,997358.0,United States,1.200000,0.0,1.200000,6.060000,258.000000,2.240000,0.000000,86.600000,2.200000,2.400000,NaN,2571.000000,5.245611,0.545821,[],1.0,0.0,1.0,0.0,2023,10,1,1,2023version,8,0,0


In [97]:
print('total sample number',len(df_user_nation_treat_m_save['version_User_Id'].unique()))
print('total sample number-old',len(df_user_nation_treat_m_save[df_user_nation_treat_m_save['version']=='2022version']['version_User_Id'].unique()))
print('total sample number-new',len(df_user_nation_treat_m_save[df_user_nation_treat_m_save['version']=='2023version']['version_User_Id'].unique()))


total sample number 680
total sample number-old 340
total sample number-new 340


In [90]:
df_user_nation_treat_3m=df_user_nation_treat_m.groupby(['version_User_Id','month_nb_3m','User_Id','nation']).agg(
    Up_vote = ('Up_vote', 'mean'),
    Down_vote = ('Down_vote', 'mean'),
    Score =('Score', 'mean'),
    pure_body_string_len =('pure_body_string_len', 'mean'),
    pure_body_smog_index =('pure_body_smog_index','mean'),
    Q_Score= ('Q_Score', 'mean'),
    Q_ViewCount= ('Q_ViewCount', 'mean'),
    Q_AnswerCount= ('Q_AnswerCount','mean'),
    Q_CommentCount= ('Q_CommentCount','mean'),
    Q_FavoriteCount= ('Q_FavoriteCount', 'mean'),
    Q_body_string_len= ('Q_body_string_len', 'mean'),
    ans_ques_time= ('ans_ques_time','mean'),
    full_code_string_ratio= ('full_code_string_ratio','mean'),     
    Q_Tags_set_filter =( 'Q_Tags_set_filter', lambda x:list(set(chain(*x)))),    

     TotalComment=('TotalComment',  'sum'),
     AnswerComment=('AnswerComment', 'sum'),
     QuestionerComment=('QuestionerComment', 'sum'),
     ViewerComment=('ViewerComment', 'sum'),    
            year=('year','max'), 
            month=('month','max'), 
            after=('after','max'), 
            treat=('treat','max')
                        ).reset_index()
df_user_nation_treat_3m['version']=df_user_nation_treat_3m['version_User_Id'].apply(lambda x:x.split('_')[0])
df_user_nation_treat_3m = df_user_nation_treat_3m.groupby('version_User_Id').filter(lambda x: set(range(1, 9)).issubset(x['month_nb_3m']))
df_user_nation_treat_3m = df_user_nation_treat_3m.groupby('User_Id').filter(lambda x: set(['2022version','2023version']).issubset(x['version']))
df_user_nation_treat_3m['Q_Tags_len_filter']=df_user_nation_treat_3m['Q_Tags_set_filter'].map(lambda x:len(x))
df_user_nation_treat_3m.loc[df_user_nation_treat_3m['Q_Tags_len_filter'] <= 1, 'Q_Tags_len_filter'] = 1

# 首先，计算每个月的Id_count均值
median_Q_Tags_len_filter = df_user_nation_treat_3m.groupby('month_nb_3m')['Q_Tags_len_filter'].transform('median')


# 接下来，比较version_User_Id和均值，高于均值则high_Id_count为1，否则为0
df_user_nation_treat_3m['median_Q_Tags_len_filter'] = (df_user_nation_treat_3m['Q_Tags_len_filter'] > median_Q_Tags_len_filter).astype(int)

df_user_nation_treat_3m.to_csv('pans/0606-df_user_nation_treat_3m.csv',encoding='utf-8-sig')

df_user_nation_treat_3m

,version_User_Id,month_nb_3m,User_Id,nation,Up_vote,Down_vote,Score,pure_body_string_len,pure_body_smog_index,Q_Score,Q_ViewCount,Q_AnswerCount,Q_CommentCount,Q_FavoriteCount,Q_body_string_len,ans_ques_time,full_code_string_ratio,Q_Tags_set_filter,TotalComment,AnswerComment,QuestionerComment,ViewerComment,year,month,after,treat,version,Q_Tags_len_filter,median_Q_Tags_len_filter
629,2022version_10020419,1,10020419.0,Deutschland,2.083333,0.000000,2.083333,1025.750000,9.716667,1.000000,1563.333333,1.083333,2.166667,0.0,3942.083333,7.203750,0.257624,[],20.0,7.0,13.0,0.0,2021,12,0,0,2022version,1,0
630,2022version_10020419,2,10020419.0,Deutschland,2.750000,0.066667,2.683333,604.500000,5.483333,1.433333,1352.983333,1.483333,0.666667,0.0,2006.166667,5.599231,0.575419,[lua],20.0,7.0,12.0,1.0,2021,5,0,0,2022version,1,0
631,2022version_10020419,3,10020419.0,Deutschland,1.750000,0.000000,1.750000,265.750000,6.125000,0.500000,428.500000,2.000000,1.750000,NaN,938.000000,4.441528,0.318917,[],8.0,4.0,4.0,0.0,2021,8,0,0,2022version,1,0
632,2022version_10020419,4,10020419.0,Deutschland,2.022222,0.000000,2.022222,775.733333,9.431111,1.511111,1476.466667,1.711111,1.088889,0.0,1394.044444,9.416907,0.343799,[],12.0,4.0,7.0,1.0,2021,11,0,0,2022version,1,0
633,2022version_10020419,5,10020419.0,Deutschland,0.800000,0.000000,0.800000,549.800000,8.560000,1.200000,668.200000,1.800000,1.000000,0.0,1335.200000,9.799778,0.271322,[],4.0,2.0,2.0,0.0,2021,12,1,0,2022version,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899156,2023version_9989141,4,9989141.0,Eesti,3.888889,0.000000,3.861111,1180.500000,9.372222,0.805556,1258.277778,1.000000,0.583333,0.0,1051.416667,5.541235,0.202693,"[c++, arduino]",19.0,7.0,11.0,1.0,2022,11,0,1,2023version,2,0
899157,2023version_9989141,5,9989141.0,Eesti,1.750000,0.000000,1.416667,584.750000,11.025000,0.250000,560.750000,1.333333,1.833333,NaN,1492.833333,6.134838,0.110068,[],11.0,2.0,7.0,2.0,2023,2,1,1,2023version,1,0
899158,2023version_9989141,6,9989141.0,Eesti,1.611111,0.000000,1.611111,538.055556,9.666667,1.166667,388.555556,1.444444,1.722222,NaN,1623.500000,10.141759,0.217977,"[arduino, c++]",6.0,2.0,4.0,0.0,2023,5,1,1,2023version,2,0
899159,2023version_9989141,7,9989141.0,Eesti,1.000000,0.000000,1.000000,504.833333,9.216667,0.166667,277.833333,1.500000,1.166667,NaN,4106.166667,4.297315,0.145089,"[c, arduino, c++]",3.0,1.0,2.0,0.0,2023,8,1,1,2023version,3,1


In [92]:
len(df_user_nation_treat_3m)/8

3342.0

In [93]:
print('total sample number',len(df_user_nation_treat_3m['version_User_Id'].unique()))
print('total sample number-old',len(df_user_nation_treat_3m[df_user_nation_treat_3m['version']=='2022version']['version_User_Id'].unique()))
print('total sample number-new',len(df_user_nation_treat_3m[df_user_nation_treat_3m['version']=='2023version']['version_User_Id'].unique()))


total sample number 3342
total sample number-old 1671
total sample number-new 1671


In [62]:
dict_users_summury=dict(Counter(df_user_nation_treat_3m['User_Id']))
print('Repetitive',len([k for k,v in dict_users_summury.items() if v==16]))
print('Unique',len([k for k,v in dict_users_summury.items() if v==8]))


Repetitive 1671
Unique 2217


In [59]:
dict_users_summury

{10017265.0: 8,
 10020419.0: 16,
 10026074.0: 8,
 100297.0: 8,
 1003565.0: 8,
 10055162.0: 16,
 10063876.0: 8,
 10069313.0: 16,
 1007220.0: 16,
 10085080.0: 16,
 1008568.0: 16,
 1008999.0: 16,
 101087.0: 8,
 10112124.0: 16,
 10113238.0: 8,
 10134209.0: 16,
 101361.0: 8,
 10147399.0: 16,
 1016343.0: 16,
 10167914.0: 8,
 1018154.0: 8,
 10188661.0: 8,
 10195153.0: 16,
 10197418.0: 16,
 10209987.0: 8,
 1021504.0: 8,
 1021720.0: 8,
 1021819.0: 16,
 102205.0: 8,
 10220740.0: 16,
 1022260.0: 8,
 1023553.0: 16,
 10262805.0: 16,
 10270590.0: 8,
 10275943.0: 8,
 1028230.0: 16,
 10289265.0: 16,
 1029088.0: 8,
 102937.0: 8,
 10304804.0: 8,
 1030675.0: 16,
 10315665.0: 8,
 103167.0: 16,
 1031689.0: 8,
 1032372.0: 8,
 1032890.0: 8,
 10335.0: 16,
 10336767.0: 8,
 10347145.0: 16,
 10348758.0: 8,
 10351316.0: 8,
 10362396.0: 8,
 10366303.0: 8,
 1037345.0: 8,
 10373693.0: 8,
 1038015.0: 16,
 10386713.0: 8,
 10397.0: 16,
 10407448.0: 8,
 10409567.0: 8,
 104149.0: 16,
 10415047.0: 16,
 1041641.0: 16,
 104